<a href="https://colab.research.google.com/github/AjeetSingh02/Notebooks/blob/master/profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install 'pandas_profiling==2.3.0'

https://stackoverflow.com/questions/1773805/how-can-i-parse-a-yaml-file-in-python

https://stackoverflow.com/questions/12470665/how-can-i-write-data-in-yaml-format-in-a-file


In [0]:
#Variable.S_TYPE_CORR
#it means that this variable is highly correlated with some variable

#the name of the variable is given by "correlation_var"
#and the correlation is given by "correlation"

In [0]:
import pandas_profiling
import pandas as pd
import numpy as np
import json
import yaml

#function to convert np.int64 into integer
def convert(o):
    if isinstance(o, np.int64): 
      return int(o) 
    elif isinstance(o, np.bool_):
      if o == True:
        return True
      return False

def save_json(csv_file, output_file_path, yaml_file):
  df = pd.read_csv(csv_file)
  profile = pandas_profiling.ProfileReport(df)

  parent_dict = profile.get_description()

  data = {}
  data["General Information"] = parent_dict["table"]
  variables = parent_dict["variables"]

  with open(yaml_file, 'r') as stream:
    try:
        possible_keys = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        return f"following error occured while loading yaml file: {exc}"

  data["variables"] = {}
  data["variables"]["header"] = possible_keys


  data["freq"] = {}
  data["freq"]["value_counts_with_nan"] = {}
  data["freq"]["value_counts_without_nan"] = {}

  #For info of each variable
  for variable, dict_ in variables.items():
    temp_list = []
    for key in possible_keys:
      if key in dict_:
        if key == "type":
          temp_list.append(dict_[key].value)
        else:  
          temp_list.append(dict_[key])
      else:
        temp_list.append("None")
    data["variables"][variable] = temp_list


  #for frequency
  with_nan = dict_["value_counts_with_nan"]  
  without_nan = dict_["value_counts_without_nan"]

  data["freq"]["value_counts_with_nan"][variable] = [list(with_nan.index), list(with_nan.values)]
  data["freq"]["value_counts_without_nan"][variable] = [list(without_nan.index), list(without_nan.values)]


  correlations = parent_dict["correlations"]

  data["correlations"] = {}

  for corr, df_ in correlations.items():
    data["correlations"][corr] = [list(df.index), [list(val) for val in df_.values]]


  #Saving at output_path
  with open(f"{output_file_path}output.json", "w") as write_file:
      json.dump(data, write_file, default=convert)  

In [0]:
csv_file = "/content/train.csv"
output_file_path = ""
yaml_file = "data.yml"
save_json(csv_file, output_file_path, yaml_file)

In [0]:
possible_keys = ['distinct_count_with_nan', 'distinct_count_without_nan', 'type', 'count', 'distinct_count', 'p_missing', 'n_missing', 'p_infinite', 'n_infinite', 
            'is_unique', 'mode', 'p_unique', 'memorysize', 'top', 'freq', 'max_length', 'mean_length', 'min_length', 'composition', 'date_warning', 'varname', 
            'varid', 'row_classes', 'mean', 'std', 'variance', 'min', 'max', 'kurtosis', 'skewness', 'sum', 'mad', 'n_zeros', 'range', 
            '5%', '25%', '50%', '75%', '95%', 'iqr', 'cv', 'p_zeros', 'correlation_var', 'correlation']

with open('data.yml', 'w') as outfile:
    yaml.dump(possible_keys, outfile, default_flow_style=False)